In [ ]:
using CairoMakie
using Arya
using LilGuys

In [ ]:
using OrderedCollections

In [ ]:
function load_profiles(filenames)

    profiles = OrderedDict{String, LilGuys.StellarProfile}()

    for (key, filename) in filenames
        profiles[key] = LilGuys.StellarProfile("density_profiles/" * filename * "_profile.toml")

    end
    
    return profiles
end

In [ ]:
log_r_label = "log r / arcmin"

In [ ]:
function plot_densities(profiles; sequential=false,
        normalize=false,
		limits=((-1, 2), (-3, 2)),
        title=""
    )
   	fig = Figure()

    if normalize
		ylabel=L"\log \Sigma\ /\ \textrm{fraction arcmin^{-2}}"
    else
        ylabel=L"\log \Sigma\ /\ \textrm{stars arcmin^{-2}}"
    end
    
    ax = Axis(fig[1, 1],
		ylabel=ylabel,
		xlabel=log_r_label,
        limits = limits,
        title=title
    )

    for (i, (key, profile)) in enumerate(profiles)
        if sequential
            kwargs = (; color=i, colorrange=(1, length(profiles)))
        else
            kwargs = (; )
        end

        if normalize
            dy = -log10(profile.M_in[end])
        else
            dy = 0
        end
        
        lines!(ax, profile.log_r, profile.log_Sigma .+ dy;
            label=key, kwargs...)
        
        if i == 1
            errscatter!(ax, profile.log_r, profile.log_Sigma .+ dy, 
                yerr=profile.log_Sigma_err)
        end
    end

    axislegend(position=:lb)

    fig
end

In [ ]:
function plot_Γ(profiles)
   	fig = Figure()
	ax = Axis(fig[1, 1],
		ylabel=L"\Gamma",
		xlabel=log_r_label,
		limits=((-1, 2), (-5, 2))
	)

    for (i, (key, profile)) in enumerate(profiles)
        lines!(ax, profile.log_r, profile.Gamma, 
                label=key)

        
        if i == 1
            errscatter!(ax, profile.log_r, profile.Gamma, 
                yerr=profile.Gamma_err)

        end
    end

    axislegend()

    fig
end

In [ ]:
function plot_Γ_max(profiles)
   	fig = Figure()
	ax = Axis(fig[1, 1],
		ylabel=L"\Gamma_\textrm{max}",
		xlabel=log_r_label,
		limits=((-1, 2), (-2, 3))
	)

    for (i, (key, profile)) in enumerate(profiles)
        
        lines!(ax, profile.log_r, profile.Gamma_max, 
            label=key)

        if i == 1
            errscatter!(ax, profile.log_r, profile.Gamma_max,
                yerr=profile.Gamma_max_err)
        end

    end

    axislegend(position=:rb)

    fig
end

### Basic uncertainties

In [ ]:
profiles = load_profiles(OrderedDict(
    "PSAT > 0.2" => "fiducial",
    "simple" => "simple",
    "no qso cand." => "noqso",
    "circular" => "psat_circ",
        ))

In [ ]:
plot_densities(profiles, limits=(-1, 2.4, -4, 1.5))

In [ ]:
plot_Γ(profiles)

In [ ]:
plot_Γ_max(profiles)

### Magnitude effect

In [ ]:
sum(profiles["G = 19.5 - 20"].counts)

In [ ]:
profiles = load_profiles(OrderedDict(
    "G = 10 - 19" => "G_10_19",
    "G = 19 - 19.5" => "G_19_19.5",
    "G = 19.5 - 20" => "G_19.5_20",
    "G = 20 - 20.5" => "G_20_20.5",
    "G = 20.5 - 21.5" => "G_20.5_21.5",
        ))

In [ ]:
plot_densities(profiles, sequential=true)

In [ ]:
plot_Γ(profiles)

In [ ]:
plot_Γ_max(profiles)

### PSAT effects